# Chapter 7. Function Decorators and Closures
## 7.1 Decorators 101

In [1]:
# Ex 7-1 : A decorator usually replaces a function with a different one
def deco(func):
    def inner():
        print('running inner()')
    return inner

In [2]:
import numpy as np

In [3]:
@deco
def target():
    print('running target()')

In [4]:
target()

running inner()


## 7.2 When Python Executes Decorators

In [14]:
# Ex 7-2 : The registration.py module
registry = []
def register(func):
    print('running register(%s)'% func)
    registry.append(func)
    return func

In [15]:
@register
def f1():
    print('running f1()')
    
@register
def f2():
    print('running f2()')


def f3():
    print('running f3()')

def main():
    print('running main()')
    print('registry ->', registry)
    f1()
    f2()
    f3()
if __name__ == '__main__':
    main()

running register(<function f1 at 0x7f1edc02f9d8>)
running register(<function f2 at 0x7f1edc02f840>)
running main()
registry -> [<function f1 at 0x7f1edc02f9d8>, <function f2 at 0x7f1edc02f840>]
running f1()
running f2()
running f3()


## 7.3 Decorator-Enhanced Strategy Pattern

In [3]:
# Example 7-3 : The promos list is filled by the promotion decorator
promos = []

def promotion(promo_func):
    promos.append(promo_func)
    return promo_func

@promotion
def fidelity(order):
    """5% discount for customers with 1000 or more fidelity points"""
    return order.total()*.05 if order.customer.fidelity>=1000 else 0

@promotion
def bulk_item(order):
    """10% discount for each LineItem with 20 or more units"""
    disocunt = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total()*.1
    return discount

@promotion
def large_order(order):
    """7% discount for orders with 10 or more distinct items"""
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total()*.07
    return 0

def best_promo(order):
    """Select best discount available"""
    return max(promo(order) for promo in promos)

## 7.4 Variable Scope Rules

## 7.5 Closures

In [31]:
from average_oo import Averager

In [32]:
avg = Averager()

In [33]:
avg(10)

10.0

In [34]:
avg(11)

10.5

In [56]:
from average import make_averager

In [60]:
# I get an error if this part is not included (i.e. defined outside the notebook).
""""""
def make_averager():
        series = []

        def averager(new_value):
                series.append(new_value)
                total = sum(series)
                return total/len(series)

        return averager


In [61]:
avg = make_averager()

In [62]:
avg(10)

10.0

In [63]:
avg(14)

12.0

In [65]:
# Ex. 7-11 : Inspecting the function created by make_averager in Example 7-9
avg.__code__.co_varnames

('new_value', 'total')

In [66]:
avg.__code__.co_freevars

('series',)

In [67]:
avg.__closure__

(<cell at 0x1106d6168: list object at 0x1106c4d88>,)

## 7.6 The nonlocal Declaration

In [69]:
# Ex 7-13 : A broken higher-order function to calculate a running average without keeping all history
def make_averager():
    count = 0
    total = 0
    
    def averager(new_value):
        count += 1
        total += new_value
        return total/count
    return averager


In [71]:
avg = make_averager()

In [72]:
avg(10)

UnboundLocalError: local variable 'count' referenced before assignment

In [73]:
# Ex 7-14: Calculate a running average wihtout keeping all history
def make_averager():
    count = 0
    total = 0
    
    def averager(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return total/count